In [1]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install faiss-gpu
!pip install einops
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub
!pip install sentence-transformers
!pip install tqdm
!pip install rouge_score nltk bert_score

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/MyDrive/DATASCI266/project')

Mounted at /content/drive


In [3]:
import os
import torch
import warnings
from datasets import load_dataset
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer , AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline, BitsAndBytesConfig
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [4]:
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel

In [5]:
# Evaluator
summary_evaluator = SummaryEvaluator()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Load the XSum dataset
data_manager = DatasetManager(sample_size=200)
dataset = data_manager.load_sampled_dataset(dataset_label="test")

# Initialize the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
t5_pipe = pipeline(
    "summarization",
    model=t5model,
    tokenizer=t5tokenizer,
    max_length=1000,
    truncation=True,
    device=0
)
#t5_pipe.model.config.pad_token_id = t5_pipe.model.config.eos_token_id

In [11]:
t5_llm_lc = HuggingFacePipeline(pipeline=t5_pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [12]:
class RAG_Summarization_Model:
  def __init__(self, chunk_size=200, chunk_overlap=50):
    self.vector_store = None
    self.rag_chain = None
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap

  def fit(self, dataset):
    # Function to split documents and include metadata
    def create_documents_with_metadata(dataset, chunk_size=200, chunk_overlap=50):
      # Initialize the text splitter
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size,  # Adjust based on your needs
          chunk_overlap=chunk_overlap  # Adjust based on your needs
      )
      documents = []
      for example in dataset:
          text = example['document']
          # Split the text into chunks while including metadata
          chunks = text_splitter.split_text(text)
          for chunk in chunks:
              documents.append(Document(page_content=chunk, metadata={"id": example['id']}))
      return documents
    # Create documents with metadata
    documents = create_documents_with_metadata(dataset, self.chunk_size, self.chunk_overlap)
    # Create a FAISS vector store
    self.vector_store = FAISS.from_documents(documents, embeddings)

    rag_template = """summarize: {context}"""

    rag_prompt = ChatPromptTemplate.from_template(rag_template)

    self.rag_chain = (
        {"context": lambda inputs: self.format_docs(self.query_vector_store(inputs['query'], inputs['id'], k=inputs['k'], fetch_k=inputs['fetch_k'])),
        "question": RunnablePassthrough()}
        | rag_prompt
        | t5_llm_lc
    )

  def query_vector_store(self, query_text, summary_id, k=3, fetch_k=20):
    # Embed the query text
    query_vector = embeddings.embed_query(query_text)
    # Search the vector store
    docs = self.vector_store.similarity_search_by_vector(query_vector, filter=dict(id=summary_id), k=k, fetch_k=fetch_k)
    return docs

  def format_docs(self, docs):
      return "\n\n".join(doc.page_content for doc in docs)

  def predict(self, dataset, k, fetch_k):
    results = []
    i = 1
    for example in tqdm(dataset):
        print(i)
        text = example['document']
        sentences = re.split(r'[.?!]\s+', text)
        example_query = sentences[0]
        #summary_w_prompt = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        summary = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        #summary = summary_w_prompt.split("summarize: ",1)[1]
        results.append(summary)
        i += 1
    return results

In [13]:
references = [example['summary'] for example in dataset]

In [14]:
import re

In [15]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=50)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 1000, but your input_length is only 117. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)


1


  0%|          | 1/200 [00:26<1:28:47, 26.77s/it]Your max_length is set to 1000, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


2


  1%|          | 2/200 [00:41<1:05:38, 19.89s/it]Your max_length is set to 1000, but your input_length is only 144. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)


...

 99%|█████████▉| 198/200 [1:01:50<00:45, 22.72s/it]Your max_length is set to 1000, but your input_length is only 75. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


199


100%|█████████▉| 199/200 [1:02:05<00:20, 20.27s/it]Your max_length is set to 1000, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


200


100%|██████████| 200/200 [1:02:20<00:00, 18.70s/it]


In [ ]:
print("chunk_size=200, chunk_overlap=50")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=50
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1857555449342186, 'rouge2': 0.025340232400528274, 'rougeL': 0.13058339249601336}
Average BLEU score: 0.016658304874744564


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5039858818054199
Average VECTOR_SIMILARITY score: 0.4676288366317749
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.19389895282765424, 'rouge2': 0.026993263967163644, 'rougeL': 0.13390560375866467}
Average BLEU score: 0.015642627355602207


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5058295726776123
Average VECTOR_SIMILARITY score: 0.46941375732421875


{'rouge': {'rouge1': 0.19389895282765424,
  'rouge2': 0.026993263967163644,
  'rougeL': 0.13390560375866467},
 'bleu': 0.015642627355602207,
 'bertscore': 0.5058295726776123,
 'vector_similarity': 0.46941375732421875}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Your max_length is set to 1000, but your input_length is only 117. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)


1


  0%|          | 1/200 [00:15<52:38, 15.87s/it]Your max_length is set to 1000, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


2


  1%|          | 2/200 [00:30<49:33, 15.02s/it]Your max_length is set to 1000, but your input_length is only 144. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)


...

 99%|█████████▉| 198/200 [1:01:34<00:39, 19.89s/it]Your max_length is set to 1000, but your input_length is only 75. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


199


100%|█████████▉| 199/200 [1:01:48<00:18, 18.28s/it]Your max_length is set to 1000, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


200


100%|██████████| 200/200 [1:02:02<00:00, 18.61s/it]


In [ ]:
print("chunk_size=200, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1876221364231862, 'rouge2': 0.026128717876996634, 'rougeL': 0.13078302541503026}
Average BLEU score: 0.01647070014493064


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4988216459751129
Average VECTOR_SIMILARITY score: 0.4638444781303406
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.18974155072399315, 'rouge2': 0.02549870206163818, 'rougeL': 0.13419483740069518}
Average BLEU score: 0.01595697849612014


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5021682381629944
Average VECTOR_SIMILARITY score: 0.4668318033218384
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.18961218074569594, 'rouge2': 0.02662032043525747, 'rougeL': 0.13069506826530175}
Average BLEU score: 0.015703701824474527


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5042239427566528
Average VECTOR_SIMILARITY score: 0.4721079170703888
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.19103413654240392, 'rouge2': 0.025707431624921676, 'rougeL': 0.1320913317958162}
Average BLEU score: 0.01551251371885335


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5044363737106323
Average VECTOR_SIMILARITY score: 0.47106048464775085


{'rouge': {'rouge1': 0.19103413654240392,
  'rouge2': 0.025707431624921676,
  'rougeL': 0.1320913317958162},
 'bleu': 0.01551251371885335,
 'bertscore': 0.5044363737106323,
 'vector_similarity': 0.47106048464775085}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=150, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Your max_length is set to 1000, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)


1


  0%|          | 1/200 [00:14<49:41, 14.98s/it]Your max_length is set to 1000, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


2


  1%|          | 2/200 [00:29<48:07, 14.58s/it]Your max_length is set to 1000, but your input_length is only 123. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


...

 99%|█████████▉| 198/200 [1:00:28<00:44, 22.08s/it]Your max_length is set to 1000, but your input_length is only 67. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)


199


100%|█████████▉| 199/200 [1:00:43<00:19, 19.83s/it]Your max_length is set to 1000, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


200


100%|██████████| 200/200 [1:00:56<00:00, 18.28s/it]


In [ ]:
print("chunk_size=150, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)